In [1]:
maxD = 5
inTime = 234
otTime = 474

In [2]:
import sys
sys.path.append("C:\Program Files\Tinysoft\Analyse.NET")
sys.path.append(r"C:\Users\xudong\Documents\github\coresearch\funcs")
import pymongo
import numpy as np
import pickle
import time
import datetime
import copy
import pandas as pd
from operator import itemgetter
import TSLPy3 as tsl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import talib
import importlib
import rschLib
np.set_printoptions(formatter={'float_kind': "{:.6f}".format})
client = pymongo.MongoClient('localhost', 27017)
db = client.quanLiang
dbt = client.tinySoftData
dtes, tkrs, name, open_mtx, high_mtx, low_mtx, close_mtx,belong, shenwan1, shenwan2, shenwan3, vol_mtx, amount_mtx = rschLib.loadDailyBarMtx()
# get time labels
# get trades


In [3]:
strategy_names = [x['file'] for x in rschLib.tagDict().values()]
offStarts = [x['off_start'] for x in rschLib.tagDict().values()]
importlib.reload(rschLib)

<module 'rschLib' from 'C:\\Users\\xudong\\Documents\\github\\coresearch\\funcs\\rschLib.py'>

In [4]:
def analyzeStrategy(strategy_name, offStart, dtes, name, tkrs):
    timeAsFloat, timeLabels, maxM, dayOff = rschLib.getTimeLabels(maxD)
    trades, tradesUsed, Po, r = rschLib.getTrades(strategy_name, name, tkrs, dtes, maxD, maxM)
    # get trade samples by good/bad trades
    tradeArea=[inTime,otTime]
    result = rschLib.getTradeAnalysisSampleGroups(r, tradeArea)

    # draw price change
    rschLib.drawPriceChange(r, strategy_name, timeLabels=timeLabels, tp=tradeArea)
    rschLib.drawPriceChange(result['rGood10'], strategy_name, timeLabels=timeLabels, title='盈利前10%交易', tp=tradeArea)
    rschLib.drawPriceChange(result['rGood20'], strategy_name, timeLabels=timeLabels, title='盈利前20%交易', tp=tradeArea)
    rschLib.drawPriceChange(result['rGood30'], strategy_name, timeLabels=timeLabels, title='盈利前30%交易', tp=tradeArea)
    rschLib.drawPriceChange(result['rBad10'], strategy_name, timeLabels=timeLabels, title='亏损前10%交易', tp=tradeArea)
    rschLib.drawPriceChange(result['rBad20'], strategy_name, timeLabels=timeLabels, title='亏损前20%交易', tp=tradeArea)
    rschLib.drawPriceChange(result['rBad30'], strategy_name, timeLabels=timeLabels, title='亏损前30%交易', tp=tradeArea)
    # analyze tags
    rschLib.analyzeTradeTags(trades, result['rGood10'], result['idxGood10'], '盈利前10%交易',strategy_name, dtes, name, offStart)
    rschLib.analyzeTradeTags(trades, result['rGood20'], result['idxGood20'], '盈利前20%交易',strategy_name, dtes, name, offStart)
    rschLib.analyzeTradeTags(trades, result['rGood30'], result['idxGood30'], '盈利前30%交易',strategy_name, dtes, name, offStart)
    rschLib.analyzeTradeTags(trades, result['rBad10'], result['idxBad10'], '亏损前10%交易',strategy_name, dtes, name, offStart)
    rschLib.analyzeTradeTags(trades, result['rBad20'], result['idxBad20'], '亏损前20%交易',strategy_name, dtes, name, offStart)
    rschLib.analyzeTradeTags(trades, result['rBad30'], result['idxBad30'], '亏损前30%交易',strategy_name, dtes, name, offStart)

    #get tag names
    tnames, idxOverLapTagList=rschLib.analyzeTradeTags(trades, r, list(range(len(trades))), '所有交易',strategy_name, dtes, name, offStart)

    #draw pnl and tag pnl
    importlib.reload(rschLib)
    [dtesByTrade, pnlByTrade]=rschLib.getPnl(dtes,tkrs, name, trades, inTime, otTime, dayOff, timeAsFloat, toDatabase='yes', strategy_name=strategy_name)
    [dtesPnlAggr,pnlAggr, numTrades] = rschLib.aggregatePnlAndDtes(dtesByTrade, pnlByTrade)
    rschLib.drawPNL(dtesPnlAggr, pnlAggr, dtes, strategy_name, toDatabase='yes')
    for i in range(len(tnames)):
        tagName = tnames[i]
        [dtesWithTag, pnlWithTag,n] = rschLib.aggregatePnlAndDtes(dtesByTrade[idxOverLapTagList[i]],pnlByTrade[idxOverLapTagList[i]])
        rschLib.drawPNL(dtesWithTag, pnlWithTag, dtes, strategy_name, toDatabase='yes', dateStart=dtesPnlAggr[0], pnlType=tagName)
        rschLib.drawPNL(dtesWithTag, pnlWithTag, dtes, strategy_name+'+'+tagName, toDatabase='yes', dateStart=dtesPnlAggr[0], pnlType='pnl')

    #analysis of number of trades vs performance
    importlib.reload(rschLib)
    rschLib.pnlVsNumtrades(pnlAggr, numTrades, strategy_name, toDatabase='yes')

In [ ]:
importlib.reload(rschLib)
for (i,strategy_name) in enumerate(strategy_names):
    offStart = offStarts[i]
    analyzeStrategy(strategy_name, offStart, dtes, name, tkrs)

10000 / 12941 1200 2020-07-14 01:31:00 2020-07-20 07:00:00
['净利润增速连续三年大于20%', '季度净利润连续三年大于1亿', '跳空高开', '突破1年新高(开盘)', '突破3个月新高(开盘)', '均线多头排列', 'MACD的DIFF高于DEA', 'MACD的DIFF大于0', '交易量放大2倍', '流通市值大于100亿', '流通市值大于250亿', '流通市值大于500亿', '流通市值小于100亿', '流通市值小于50亿', '流通市值小于25亿']
盈利前10%交易 净利润增速连续三年大于20% 2.84 294402.0
盈利前10%交易 季度净利润连续三年大于1亿 6.56 688398.0
盈利前10%交易 跳空高开 18.82 287743.0
盈利前10%交易 突破1年新高(开盘) 41.36 107168
盈利前10%交易 突破3个月新高(开盘) 41.36 163663
盈利前10%交易 均线多头排列 97.16 3237424.0
盈利前10%交易 MACD的DIFF高于DEA 2.3 411930.0
盈利前10%交易 MACD的DIFF大于0 99.67 5660481
盈利前10%交易 交易量放大2倍 11.82 674557.0
盈利前10%交易 流通市值大于100亿 18.05 1472681.0
盈利前10%交易 流通市值大于250亿 7.11 518397.0
盈利前10%交易 流通市值大于500亿 3.17 220378.0
盈利前10%交易 流通市值小于100亿 81.95 16441463.0
盈利前10%交易 流通市值小于50亿 69.69 14868750.0
盈利前10%交易 流通市值小于25亿 50.33 12680931.0
['净利润增速连续三年大于20%', '季度净利润连续三年大于1亿', '跳空高开', '突破1年新高(开盘)', '突破3个月新高(开盘)', '均线多头排列', 'MACD的DIFF高于DEA', 'MACD的DIFF大于0', '交易量放大2倍', '流通市值大于100亿', '流通市值大于250亿', '流通市值大于500亿', '流通市值小于100亿', '流通市值小于50亿', '流通市值小于25亿']


C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:187: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:187: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:187: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the r

C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:45: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:45: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


['净利润增速连续三年大于20%', '季度净利润连续三年大于1亿', '跳空高开', '突破1年新高(开盘)', '突破3个月新高(开盘)', '均线多头排列', 'MACD的DIFF高于DEA', 'MACD的DIFF大于0', '交易量放大2倍', '流通市值大于100亿', '流通市值大于250亿', '流通市值大于500亿', '流通市值小于100亿', '流通市值小于50亿', '流通市值小于25亿']
盈利前10%交易 净利润增速连续三年大于20% 3.52 294402.0
盈利前10%交易 季度净利润连续三年大于1亿 7.17 688398.0
盈利前10%交易 跳空高开 17.26 287743.0
盈利前10%交易 突破1年新高(开盘) 15.91 107168
盈利前10%交易 突破3个月新高(开盘) 28.66 163663
盈利前10%交易 均线多头排列 94.91 3237424.0
盈利前10%交易 MACD的DIFF高于DEA 1.8 411930.0
盈利前10%交易 MACD的DIFF大于0 98.6 5660481
盈利前10%交易 交易量放大2倍 11.31 674557.0
盈利前10%交易 流通市值大于100亿 15.46 1472681.0
盈利前10%交易 流通市值大于250亿 5.68 518397.0
盈利前10%交易 流通市值大于500亿 2.52 220378.0
盈利前10%交易 流通市值小于100亿 84.54 16441463.0
盈利前10%交易 流通市值小于50亿 72.87 14868750.0
盈利前10%交易 流通市值小于25亿 51.51 12680931.0
['净利润增速连续三年大于20%', '季度净利润连续三年大于1亿', '跳空高开', '突破1年新高(开盘)', '突破3个月新高(开盘)', '均线多头排列', 'MACD的DIFF高于DEA', 'MACD的DIFF大于0', '交易量放大2倍', '流通市值大于100亿', '流通市值大于250亿', '流通市值大于500亿', '流通市值小于100亿', '流通市值小于50亿', '流通市值小于25亿']
盈利前20%交易 净利润增速连续三年大于20% 3.74 294402.0
盈利前20%交易 季度净利润连续三年大于1亿

C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:187: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:187: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:187: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the r

...

C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:45: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:45: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:45: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcPa

['净利润增速连续三年大于20%', '季度净利润连续三年大于1亿', '跳空高开', '突破1年新高(开盘)', '突破3个月新高(开盘)', '均线多头排列', 'MACD的DIFF高于DEA', 'MACD的DIFF大于0', '交易量放大2倍', '流通市值大于100亿', '流通市值大于250亿', '流通市值大于500亿', '流通市值小于100亿', '流通市值小于50亿', '流通市值小于25亿']
盈利前10%交易 净利润增速连续三年大于20% 3.07 294402.0
盈利前10%交易 季度净利润连续三年大于1亿 9.14 688398.0
盈利前10%交易 跳空高开 6.12 287743.0
盈利前10%交易 突破1年新高(开盘) 1.41 107168
盈利前10%交易 突破3个月新高(开盘) 3.72 163663
盈利前10%交易 均线多头排列 36.64 3237424.0
盈利前10%交易 MACD的DIFF高于DEA 99.97 411930.0
盈利前10%交易 MACD的DIFF大于0 36.29 5660481
盈利前10%交易 交易量放大2倍 17.84 674557.0
盈利前10%交易 流通市值大于100亿 15.46 1472681.0
盈利前10%交易 流通市值大于250亿 6.06 518397.0
盈利前10%交易 流通市值大于500亿 2.48 220378.0
盈利前10%交易 流通市值小于100亿 84.54 16441463.0
盈利前10%交易 流通市值小于50亿 69.34 14868750.0
盈利前10%交易 流通市值小于25亿 44.59 12680931.0
['净利润增速连续三年大于20%', '季度净利润连续三年大于1亿', '跳空高开', '突破1年新高(开盘)', '突破3个月新高(开盘)', '均线多头排列', 'MACD的DIFF高于DEA', 'MACD的DIFF大于0', '交易量放大2倍', '流通市值大于100亿', '流通市值大于250亿', '流通市值大于500亿', '流通市值小于100亿', '流通市值小于50亿', '流通市值小于25亿']
盈利前20%交易 净利润增速连续三年大于20% 3.19 294402.0
盈利前20%交易 季度净利润连续三年大于1亿

C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:187: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:187: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:187: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the r

C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:187: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:187: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:187: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the r

10000 / 1078059 1200 2018-11-12 01:31:00 2018-11-16 07:00:00
30000 / 1078059 1200 2018-11-21 01:31:00 2018-11-27 07:00:00
40000 / 1078059 1200 2018-11-26 01:31:00 2018-11-30 07:00:00
50000 / 1078059 1200 2018-11-30 01:31:00 2018-12-06 07:00:00
70000 / 1078059 1200 2018-12-12 01:31:00 2018-12-18 07:00:00
80000 / 1078059 1200 2018-12-18 01:31:00 2018-12-24 07:00:00
90000 / 1078059 1200 2019-01-02 01:31:00 2019-01-08 07:00:00
100000 / 1078059 1200 2019-01-16 01:31:00 2019-01-22 07:00:00
110000 / 1078059 1200 2019-01-23 01:31:00 2019-01-29 07:00:00
130000 / 1078059 1200 2019-02-13 01:31:00 2019-02-19 07:00:00
140000 / 1078059 1200 2019-02-19 01:31:00 2019-02-25 07:00:00
150000 / 1078059 1200 2019-02-22 01:31:00 2019-02-28 07:00:00
170000 / 1078059 1200 2019-03-01 01:31:00 2019-03-07 07:00:00
190000 / 1078059 1200 2019-03-08 01:31:00 2019-03-14 07:00:00
200000 / 1078059 1200 2019-03-13 01:31:00 2019-03-19 07:00:00
210000 / 1078059 1200 2019-03-15 01:31:00 2019-03-21 07:00:00
220000 / 107805

In [ ]:
#%load_ext line_profiler
#%lprun -f analyzeStrategy analyzeStrategy(strategy_name, offStart, dtes, name, tkrs)